In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout

Using TensorFlow backend.


In [2]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [3]:
from keras import initializers
import random
from keras.layers import LeakyReLU

In [6]:
df=pd.read_excel(r'E:\Machine learning and AI\Data for ML\DATA for ANN.xlsx')

In [7]:
vari=df.drop(labels=['stress','diflection','WIRF','CIRF','SCIRF','F','U','D','sr.no'],axis=1)
vari

,d,t,Ecz,Gyz,Gxz,Tcz,Syz,Sxz,core density,weight
0,40,0.28,137.90,75.842,144.79,0.6550,0.34474,0.58605,25.630,1.43176
1,40,0.28,310.26,111.700,220.63,1.3790,0.55158,0.93079,36.842,1.88024
2,40,0.28,482.63,146.170,296.47,2.1374,0.86184,1.37900,48.055,2.32876
3,40,0.28,723.95,179.260,379.21,2.8269,1.10320,1.72370,59.268,2.77728
4,40,0.28,930.79,199.950,448.16,3.8611,1.37900,2.13740,67.278,3.09768
...,...,...,...,...,...,...,...,...,...,...
233,10,1.68,482.63,146.170,296.47,2.1374,0.86184,1.37900,48.055,2.91991
234,10,1.68,723.95,179.260,379.21,2.8269,1.10320,1.72370,59.268,3.03204
235,10,1.68,930.79,199.950,448.16,3.8611,1.37900,2.13740,67.278,3.11214
236,10,1.68,1379.00,153.730,592.95,5.5158,1.93050,2.96470,86.500,3.30436


In [8]:
ans=df.drop(['t','d','Ecz','Gyz','Gxz','Tcz','Syz','Sxz','core density','weight','sr.no','stress','diflection','F','CIRF','SCIRF','U','D'],axis=1)
ans

,WIRF
0,0.366400
1,0.243870
2,0.195450
3,0.159650
4,0.140420
...,...
233,0.051349
234,0.050026
235,0.049401
236,0.048286


In [9]:
from sklearn.model_selection import train_test_split 
X_train1, X_test, y_train1, y_test = train_test_split(vari, ans, test_size=0.15, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train1, test_size=0.15, random_state=0)

In [13]:
train_stats = X_train.describe()
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
d,171.0,22.239766,10.487856,10.00000,12.000000,18.000000,30.000000,40.00000
t,171.0,0.887485,0.489744,0.28000,0.560000,0.840000,1.120000,1.68000
Ecz,171.0,791.289006,553.000016,137.90000,310.260000,723.950000,1379.000000,1827.10000
Gyz,171.0,162.757754,65.732153,75.84200,111.700000,153.730000,199.950000,299.92000
Gxz,171.0,388.444035,188.940256,144.79000,220.630000,379.210000,592.950000,723.95000
Tcz,171.0,3.185325,2.066808,0.65500,1.379000,2.826900,5.515800,6.89480
Syz,171.0,1.181596,0.681629,0.34474,0.551580,1.103200,1.930500,2.41320
Sxz,171.0,1.857552,1.033071,0.58605,0.930790,1.723700,2.964700,3.75760
core density,171.0,59.436930,25.392957,25.63000,36.842000,59.268000,86.500000,104.12000
weight,171.0,2.607960,1.132637,0.66286,1.707595,2.430684,3.306275,6.60416


In [14]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(X_train)
normed_test_data = norm(X_test)
normed_val_data = norm(X_val)

In [15]:
df1= pd.DataFrame(columns=['activation','seed','neurons per layer','LR','MAE'])
acti1=['relu','tanh','softmax']

In [ ]:
for a in range (0,3):    
    for b in range (2,61,4):
        def build_model(c,d):
            tf.random.set_seed(b)
            model = keras.Sequential([
                layers.Dense(c,activation=acti1[a], input_shape=[len(vari.keys())]),
                layers.Dense(c,activation=acti1[a]),
                layers.Dense(c,activation=acti1[a]),
                layers.Dense(c,activation=acti[a]),
                layers.Dense(1)
              ])

            optimizer = tf.keras.optimizers.RMSprop(1/(10)**d)

            model.compile(loss='mse',
                            optimizer=optimizer,
                            metrics=['mae', 'mse'])
            return model

        for c in range (32,961,128):
            for d in range  (3,5):
                model = build_model(c,d)
                EPOCHS = 1000
             # The patience parameter is the amount of epochs to check for improvement
                early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
                early_history = model.fit(normed_train_data, y_train,batch_size=20, 
                epochs=EPOCHS, validation_data=(normed_val_data,y_val), shuffle=False,verbose=0, 
                callbacks=[early_stop, tfdocs.modeling.EpochDots()])
                loss, mae, mse = model.evaluate(normed_test_data, y_test, verbose=2)
                print("Testing set Mean Abs Error: {:5.6f} WIRF".format(mae))
                df1 = df1.append({'activation':acti[a],'seed':b,'neurons per layer': c,'LR': d,'MAE':mae}, ignore_index=True)

In [ ]:
df1.to_excel('Desktop\df1.xlsx')

In [ ]:
df2= pd.DataFrame(columns=['activation','seed','neurons per layer','LR','MAE'])
acti2=['softplus','elu','sigmoid']

In [ ]:
for a in range (0,3):    
    for b in range (2,61,4):
        def build_model(c,d):
            tf.random.set_seed(b)
            model = keras.Sequential([
                layers.Dense(c,activation=acti2[a], input_shape=[len(vari.keys())]),
                layers.Dense(c,activation=acti2[a]),
                layers.Dense(c,activation=acti2[a]),
                layers.Dense(c,activation=acti[a]),
                layers.Dense(1)
              ])

            optimizer = tf.keras.optimizers.RMSprop(1/(10)**d)

            model.compile(loss='mse',
                            optimizer=optimizer,
                            metrics=['mae', 'mse'])
            return model

        for c in range (32,961,128):
            for d in range  (3,5):
                model = build_model(c,d)
                EPOCHS = 1000
             # The patience parameter is the amount of epochs to check for improvement
                early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
                early_history = model.fit(normed_train_data, y_train,batch_size=20, 
                epochs=EPOCHS, validation_data=(normed_val_data,y_val), shuffle=False,verbose=0, 
                callbacks=[early_stop, tfdocs.modeling.EpochDots()])
                loss, mae, mse = model.evaluate(normed_test_data, y_test, verbose=2)
                print("Testing set Mean Abs Error: {:5.6f} WIRF".format(mae))
                df2 = df2.append({'activation':acti[a],'seed':b,'neurons per layer': c,'LR': d,'MAE':mae}, ignore_index=True)

In [ ]:
df2.to_excel('Desktop\df2.xlsx')

In [ ]:
df3= pd.DataFrame(columns=['activation','seed','neurons per layer','LR','MAE'])
acti3=['selu','softsign','hard_sigmoid','exponential']

In [ ]:
for a in range (0,4):    
    for b in range (2,61,4):
        def build_model(c,d):
            tf.random.set_seed(b)
            model = keras.Sequential([
                layers.Dense(c,activation=acti3[a], input_shape=[len(vari.keys())]),
                layers.Dense(c,activation=acti3[a]),
                layers.Dense(c,activation=acti3[a]),
                layers.Dense(c,activation=acti[a]),
                layers.Dense(1)
              ])

            optimizer = tf.keras.optimizers.RMSprop(1/(10)**d)

            model.compile(loss='mse',
                            optimizer=optimizer,
                            metrics=['mae', 'mse'])
            return model

        for c in range (32,961,128):
            for d in range  (3,5):
                model = build_model(c,d)
                EPOCHS = 1000
             # The patience parameter is the amount of epochs to check for improvement
                early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
                early_history = model.fit(normed_train_data, y_train,batch_size=20, 
                epochs=EPOCHS, validation_data=(normed_val_data,y_val), shuffle=False,verbose=0, 
                callbacks=[early_stop, tfdocs.modeling.EpochDots()])
                loss, mae, mse = model.evaluate(normed_test_data, y_test, verbose=2)
                print("Testing set Mean Abs Error: {:5.6f} WIRF".format(mae))
                df3 = df3.append({'activation':acti[a],'seed':b,'neurons per layer': c,'LR': d,'MAE':mae}, ignore_index=True)

In [ ]:
df3.to_excel('Desktop\df3.xlsx')